# Load and Prep the Data

In [1]:
import csv
import pandas as pd

In [2]:
df = pd.read_csv('allrecipes_recipes_combined.csv')
df.head()

,name,ingredients,instructions
0,Death by Chocolate Mousse,"['21 chocolate sandwich cookies, crushed', '¼ ...",Preheat oven to 350 degrees F (175 degrees C)....
1,Shrimp and Crabmeat Loaf,"['1 (6 ounce) can small shrimp, drained', '1 (...","In a bowl, combine well the shrimp, crab, mayo..."
2,Senate Bean Soup from Idahoan®,['1 (4 ounce) package Idahoan® Roasted Garlic ...,Heat olive oil in stock pot over medium high h...
3,Nutmeg Refrigerator Cookies,"['½ cup butter', '⅓ cup white sugar', '⅔ cup p...","In a large bowl, cream together the butter, wh..."
4,French Toast Sandwich,"['2 eggs', '3 tablespoons milk', '3 tablespoon...","Whisk eggs, 3 tablespoons milk, white sugar, a..."


### Some of the recipe names have '&reg;' as text and not the symbol.

In [3]:
df['name'] = df['name'].str.replace('&reg;', '')

### The Ingredients Data has been saved as a string. Convert to list.

In [4]:
import ast

def string_to_list(x):
    return ast.literal_eval(x)

In [5]:
df['ingredients'] = df['ingredients'].apply(string_to_list)
df.head()

,name,ingredients,instructions
0,Death by Chocolate Mousse,"[21 chocolate sandwich cookies, crushed, ¼ cup...",Preheat oven to 350 degrees F (175 degrees C)....
1,Shrimp and Crabmeat Loaf,"[1 (6 ounce) can small shrimp, drained, 1 (6 o...","In a bowl, combine well the shrimp, crab, mayo..."
2,Senate Bean Soup from Idahoan®,[1 (4 ounce) package Idahoan® Roasted Garlic F...,Heat olive oil in stock pot over medium high h...
3,Nutmeg Refrigerator Cookies,"[½ cup butter, ⅓ cup white sugar, ⅔ cup packed...","In a large bowl, cream together the butter, wh..."
4,French Toast Sandwich,"[2 eggs, 3 tablespoons milk, 3 tablespoons whi...","Whisk eggs, 3 tablespoons milk, white sugar, a..."


### Need to change vulgar fractions (single character fractions) into long-form strings. ('½' to '1/2')

In [12]:
# Dictionary to map unicode fractions to expanded strings. 
# These are all of the vulgar fraction options. (Aside from one with a zero numerator.)
fraction_dict = {'½': '1/2',
                 '⅓': '1/3',
                 '⅔': '2/3',
                 '¼': '1/4',
                 '¾': '3/4',
                 '⅕': '1/5',
                 '⅖': '2/5',
                 '⅗': '3/5',
                 '⅘': '4/5',
                 '⅙': '1/6',
                 '⅚': '5/6',
                 '⅐': '1/7',
                 '⅛': '1/8',
                 '⅜': '3/8',
                 '⅝': '5/8',
                 '⅞': '7/8',
                 '⅑': '1/9',
                 '⅒': '1/10'}

In [13]:
def fraction_mapper(x):
    for key in fraction_dict:
        for i in range(len(x)):
            if key in x[i]:
                x[i] = x[i].replace(key, fraction_dict[key])
                
    return(x)

In [14]:
df['ingredients'] = df['ingredients'].apply(fraction_mapper)
df.head()

,name,ingredients,instructions
0,Death by Chocolate Mousse,"[21 chocolate sandwich cookies, crushed, 1/4 c...",Preheat oven to 350 degrees F (175 degrees C)....
1,Shrimp and Crabmeat Loaf,"[1 (6 ounce) can small shrimp, drained, 1 (6 o...","In a bowl, combine well the shrimp, crab, mayo..."
2,Senate Bean Soup from Idahoan®,[1 (4 ounce) package Idahoan® Roasted Garlic F...,Heat olive oil in stock pot over medium high h...
3,Nutmeg Refrigerator Cookies,"[1/2 cup butter, 1/3 cup white sugar, 2/3 cup ...","In a large bowl, cream together the butter, wh..."
4,French Toast Sandwich,"[2 eggs, 3 tablespoons milk, 3 tablespoons whi...","Whisk eggs, 3 tablespoons milk, white sugar, a..."


In [140]:
df['ingredients'][0]

['21 chocolate sandwich cookies, crushed',
 '1/4 cup butter, softened',
 '1 cup heavy cream',
 '1 (12 ounce) package semisweet chocolate chips',
 '1 teaspoon vanilla extract',
 '1 pinch salt',
 '2 cups heavy cream',
 '1/4 cup white sugar',
 '1 cup heavy cream, chilled',
 '1/4 cup white sugar']

### Remove ingredients that only appear once

In [15]:
from collections import Counter

In [16]:
ingredient_counter = Counter()

# Count each instance of each ingredient
for i in range(len(df)):
    for j in range(len(df['ingredients'][i])):
        ingredient = df['ingredients'][i][j]
        ingredient_counter.update({ingredient: 1})

In [17]:
# Get the ingredients that only appear once
single_ing= []
for ing, num in ingredient_counter.items():
    if num == 1:
        single_ing.append(ing)

In [18]:
# Number of ingredients that only appear once in the 70k recipes.
# These are likely incredibly specific entries.
len(single_ing)

106117

In [19]:
import datetime
# Get rid of the single-time ingredients counting backwards in each list
# so as to not go out of index range after removing one
for i in range(len(df)):
    for j in range(len(df['ingredients'][i])-1, -1, -1):
        if df['ingredients'][i][j] in single_ing:
            ingredient = df['ingredients'][i][j]
            # Remove from the ingredients
            df['ingredients'][i].remove(ingredient)
            # Remove from list to not slow down loop
            single_ing.remove(ingredient)
    if i % 2000 == 0:
        print(i, datetime.datetime.now())

0 2020-05-06 12:29:09.334246
2000 2020-05-06 12:30:07.189051
4000 2020-05-06 12:31:00.636116
6000 2020-05-06 12:31:53.772659
8000 2020-05-06 12:32:46.182365
10000 2020-05-06 12:33:33.967957
12000 2020-05-06 12:34:19.634365
14000 2020-05-06 12:35:07.979174
16000 2020-05-06 12:35:54.658555
18000 2020-05-06 12:36:36.512588
20000 2020-05-06 12:37:14.057821
22000 2020-05-06 12:37:50.136387
24000 2020-05-06 12:38:25.537389
26000 2020-05-06 12:38:58.639839
28000 2020-05-06 12:39:27.540525
30000 2020-05-06 12:39:54.136953
32000 2020-05-06 12:40:17.846529
34000 2020-05-06 12:40:39.598366
36000 2020-05-06 12:40:59.481155
38000 2020-05-06 12:41:15.119922
40000 2020-05-06 12:41:29.149427
42000 2020-05-06 12:41:41.813516
44000 2020-05-06 12:41:52.949758
46000 2020-05-06 12:42:03.390828
48000 2020-05-06 12:42:12.553317
50000 2020-05-06 12:42:20.521004
52000 2020-05-06 12:42:27.139301
54000 2020-05-06 12:42:32.781207
56000 2020-05-06 12:42:37.641208
58000 2020-05-06 12:42:41.912778
60000 2020-05-06 1

In [20]:
ingredients_len = []
for i in range(len(df)):
    ingredients_len.append(len(df['ingredients'][i]))

max(ingredients_len)

56

In [69]:
ingredients_len.index(56)

31004

In [84]:
indices = [i for i, x in enumerate(ingredients_len) if x > 30]
indices

[1619,
 4238,
 10159,
 11387,
 24655,
 31004,
 31092,
 35126,
 38444,
 48681,
 52727,
 56944,
 62148,
 62192,
 65805,
 66550]

In [85]:
for i in indices:
    print(df.iloc[i])
    print('='*30)

name                                                 Cranberry Syrup
ingredients        [4, , c, u, p, s, , c, r, a, n, b, e, r, r, y,...
instructions       Bring cranberry juice to a boil in a non-react...
ingredient_dict                     {'ingredient1': ['4', None, '']}
Name: 1619, dtype: object
name                                   Game Day Halftime Snack Board
ingredients        [1 cup mayonnaise, 4 teaspoons dry mustard pow...
instructions       Whisk mayonnaise, dry mustard, Worcestershire ...
ingredient_dict    {'ingredient1': ['1', 'cup', 'mayonnaise'], 'i...
Name: 4238, dtype: object
name                                               Watermelon Cooler
ingredients        [1, , s, m, a, l, l, , w, a, t, e, r, m, e, l,...
instructions       Blend watermelon cubes in a blender until smoo...
ingredient_dict                     {'ingredient1': ['1', None, '']}
Name: 10159, dtype: object
name                                                   Caramel Sauce
ingredients        [1, ,

In [87]:
for i in indices:
    print(df['ingredients'][i])

['4', '', 'c', 'u', 'p', 's', '', 'c', 'r', 'a', 'n', 'b', 'e', 'r', 'r', 'y', '', 'j', 'u', 'i', 'c', 'e', '', '(', 's', 'u', 'c', 'h', '', 'a', 's', '', 'c', 'e', 'a', 'n', '', 'p', 'r', 'a', 'y', '®', ')']
['1 cup mayonnaise', "4 teaspoons dry mustard powder (such as Colman's&reg;)", '1 teaspoon Worcestershire sauce', '1 teaspoon steak sauce', '1 teaspoon half-and-half', '1 teaspoon lemon juice', '1/2 teaspoon sriracha sauce', 'salt to taste', '2 tablespoons butter, divided', '12 ounces bay scallops', '1/3 cup sliced mushrooms', '1 tablespoon minced shallot', '1/2 cup half-and-half', '1 tablespoon all-purpose flour', '2 tablespoons dry white wine', '1 tablespoon dry sherry', '1/2 teaspoon lemon juice', '1/2 teaspoon Dijon mustard', '1/4 cup shredded Parmesan cheese', '1 tablespoon chopped fresh tarragon', 'ground black pepper to taste', '4 fluid ounces gin', '2 fluid ounces sweet vermouth', '2 teaspoons maraschino cherry juice', '2 dashes orange bitters', '4 maraschino cherries', '1

In [90]:
indices[::-1]

[66550,
 65805,
 62192,
 62148,
 56944,
 52727,
 48681,
 38444,
 35126,
 31092,
 31004,
 24655,
 11387,
 10159,
 4238,
 1619]

In [91]:
for i in indices[::-1]:
    print(i)

66550
65805
62192
62148
56944
52727
48681
38444
35126
31092
31004
24655
11387
10159
4238
1619


In [95]:
for i in indices[::-1]:
    df = df.drop(i, axis=0)

In [97]:
df = df.reset_index(drop=True)

In [98]:
ingredients_len = []
for i in range(len(df)):
    ingredients_len.append(len(df['ingredients'][i]))

max(ingredients_len)

30

In [99]:
indices = [i for i, x in enumerate(ingredients_len) if x > 30]
indices

[]

### Put the ingredients into a dictionary that contains values of measurement quantity, measurement unit, and ingredient ('1/4', 'cup', 'butter, softened').

In [22]:
# These are measurement units from another notebook.
measurement_units = [
 'packages', 'package', 'slices', 'slice', 
 'bags', 'bag', 'bars', 'bar', 'bottles', 'bottle', 'boxes' 'box', 'bulbs', 'bulb', 'bunches', 'bunch', 
 'cans', 'can', 'containers', 'container', 'cubes', 'cube', 'cups', 'cup', 
 'dashes', 'dash', 'drops', 'drop',
 'envelopes', 'envelope',
 'fillets', 'fillet',
 'gallons', 'gallon', 'granules', 'granule',
 'halfes', 'half', 'heads', 'head',
 'jars', 'jar',
 'layers', 'layer', 'leaf', 'leaves', 'legs', 'leg', 'links', 'link', 'loaf', 'loaves',
 'ounces', 'ounce',
 'packets', 'packet', 'pieces', 'piece', 'pinches', 'pinch', 'pints', 'pint', 'pounds', 'pound',
 'quarts', 'quart',
 'sprigs', 'sprig', 'squares', 'square', 'stalks', 'stalk', 'strips', 'strip',
 'tablespoons', 'tablespoon','teaspoons', 'teaspoon', 'thighs', 'thigh', 'trays', 'tray']

In [23]:
import re

In [26]:
def ingred_dict(x):
    '''
    This function is meant to take in a list of ingredients for a recipe.
    It then parses out the ingredients and saves the quantity of an ingredient,
    the unit of measurement for that ingredient, and the name of the ingredient.
    This information is then saved in a dictionary and returned.
    '''
    my_dict = {}  # Dictionary for the current recipe
    pattern = re.compile(r'^[\d/\s]+')  # Include white space to catch compound fractions

    for i in range(len(x)):
        matches = pattern.finditer(x[i])
        ingredient_test = x[i]

        for match in matches:

            quantity = match.group(0).strip()  # Quantity of measurement set

            ingredient_test = ingredient_test.strip(quantity)  # Save everything after removing quantity

            check = 0
            breaker = False

            pattern_2 = re.compile(r'^[(\d\s]+')  # Check for any numbers in parenthesis
            matches_2 = pattern_2.finditer(ingredient_test)

            for unit in measurement_units:
                if matches_2:  # If there's a match for a number in parenthesis
                    matches_2 = False  # Don't check this conditional again
                    continue  # Skip this unit of measurement
                elif unit in ingredient_test:
                    ingredient = ingredient_test.split(unit)[1].strip()  # Ingredient set
                    units = (ingredient_test.split(unit)[0] + unit).strip()  # Unit set (including any parenthesis before)
                    check = 1  # Set check to 1 so the last conditional doesn't execute
                    breaker = True
                if breaker == True:
                    break
            if check == 0:  # If no unit measurement is found (like the ingredient is "1 egg")
                ingredient = ingredient_test.strip()
                units = None
            
            ingred_num = f'ingredient{i+1}'
            # Save ingredient information as a list
            my_dict[ingred_num] = [quantity, units, ingredient]

    return my_dict

In [100]:
df['ingredient_dict'] = df['ingredients'].apply(ingred_dict)
df.head()

,name,ingredients,instructions,ingredient_dict
0,Death by Chocolate Mousse,"[21 chocolate sandwich cookies, crushed, 1/4 c...",Preheat oven to 350 degrees F (175 degrees C)....,"{'ingredient1': ['21', None, 'chocolate sandwi..."
1,Shrimp and Crabmeat Loaf,"[1 (6 ounce) can small shrimp, drained, 1 (6 o...","In a bowl, combine well the shrimp, crab, mayo...","{'ingredient1': ['1', '(6 ounce) can', 'small ..."
2,Senate Bean Soup from Idahoan®,[1 (4 ounce) package Idahoan® Roasted Garlic F...,Heat olive oil in stock pot over medium high h...,"{'ingredient1': ['1', '(4 ounce) package', 'Id..."
3,Nutmeg Refrigerator Cookies,"[1/2 cup butter, 1/3 cup white sugar, 2/3 cup ...","In a large bowl, cream together the butter, wh...","{'ingredient1': ['1/2', 'cup', 'butter'], 'ing..."
4,French Toast Sandwich,"[2 eggs, 3 tablespoons milk, 3 tablespoons whi...","Whisk eggs, 3 tablespoons milk, white sugar, a...","{'ingredient1': ['2', None, 'eggs'], 'ingredie..."


In [28]:
df['ingredient_dict'][0]

{'ingredient1': ['21', None, 'chocolate sandwich cookies, crushed'],
 'ingredient2': ['1/4', 'cup', 'butter, softened'],
 'ingredient3': ['1', 'cup', 'heavy cream'],
 'ingredient4': ['1', '(12 ounce) package', 'semisweet chocolate chips'],
 'ingredient5': ['1', 'teaspoon', 'vanilla extract'],
 'ingredient6': ['1', 'pinch', 'salt'],
 'ingredient7': ['2', 'cups', 'heavy cream'],
 'ingredient8': ['1/4', 'cup', 'white sugar'],
 'ingredient9': ['1', 'cup', 'heavy cream, chilled'],
 'ingredient10': ['1/4', 'cup', 'white sugar']}

In [32]:
len(df['ingredient_dict'][0])

10

In [33]:
ing = 'ingredient1'
df['ingredient_dict'][0][ing]

['21', None, 'chocolate sandwich cookies, crushed']

In [35]:
for j in range(len(df['ingredient_dict'][0])):
    print(df['ingredient_dict'][0][f'ingredient{j+1}'])

['21', None, 'chocolate sandwich cookies, crushed']
['1/4', 'cup', 'butter, softened']
['1', 'cup', 'heavy cream']
['1', '(12 ounce) package', 'semisweet chocolate chips']
['1', 'teaspoon', 'vanilla extract']
['1', 'pinch', 'salt']
['2', 'cups', 'heavy cream']
['1/4', 'cup', 'white sugar']
['1', 'cup', 'heavy cream, chilled']
['1/4', 'cup', 'white sugar']


In [101]:
col_list = ['name']
for i in range(max(ingredients_len)):
    col_list.append(f'ingredient{i+1}')

df_csv = pd.DataFrame(columns=col_list)

df_csv.head()

,name,ingredient1,ingredient2,ingredient3,ingredient4,ingredient5,ingredient6,ingredient7,ingredient8,ingredient9,...,ingredient21,ingredient22,ingredient23,ingredient24,ingredient25,ingredient26,ingredient27,ingredient28,ingredient29,ingredient30


In [102]:
for i in range(len(df)):
    new_dict = {'name': df['name'][i]}
    for j in range(len(df['ingredient_dict'][i])):
        try:
            new_dict[f'ingredient{j+1}'] = df['ingredient_dict'][i][f'ingredient{j+1}']
        except:
            continue
    df_csv = df_csv.append(new_dict, ignore_index=True)
    
    if i % 2000 == 0:
        print(i, datetime.datetime.now())

0 2020-05-06 20:38:22.407682
2000 2020-05-06 20:38:34.899265
4000 2020-05-06 20:38:49.424052
6000 2020-05-06 20:39:08.625620
8000 2020-05-06 20:39:32.630403
10000 2020-05-06 20:40:01.316667
12000 2020-05-06 20:40:36.054740
14000 2020-05-06 20:41:17.452037
16000 2020-05-06 20:42:05.576261
18000 2020-05-06 20:42:59.476128
20000 2020-05-06 20:43:59.301110
22000 2020-05-06 20:45:05.230233
24000 2020-05-06 20:46:17.038143
26000 2020-05-06 20:47:34.716903
28000 2020-05-06 20:48:58.215629
30000 2020-05-06 20:50:26.960902
32000 2020-05-06 20:52:02.414071
34000 2020-05-06 20:53:43.481171
36000 2020-05-06 20:55:30.954352
38000 2020-05-06 20:57:24.466813
40000 2020-05-06 20:59:23.299151
42000 2020-05-06 21:01:27.354897
44000 2020-05-06 21:03:37.028013
46000 2020-05-06 21:05:52.303806
48000 2020-05-06 21:08:13.097173
50000 2020-05-06 21:10:39.549821
52000 2020-05-06 21:13:10.948458
54000 2020-05-06 21:15:47.693428
56000 2020-05-06 21:18:29.880049
58000 2020-05-06 21:21:17.623781
60000 2020-05-06 2

In [103]:
df_csv.head()

,name,ingredient1,ingredient2,ingredient3,ingredient4,ingredient5,ingredient6,ingredient7,ingredient8,ingredient9,...,ingredient21,ingredient22,ingredient23,ingredient24,ingredient25,ingredient26,ingredient27,ingredient28,ingredient29,ingredient30
0,Death by Chocolate Mousse,"[21, None, chocolate sandwich cookies, crushed]","[1/4, cup, butter, softened]","[1, cup, heavy cream]","[1, (12 ounce) package, semisweet chocolate ch...","[1, teaspoon, vanilla extract]","[1, pinch, salt]","[2, cups, heavy cream]","[1/4, cup, white sugar]","[1, cup, heavy cream, chilled]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Shrimp and Crabmeat Loaf,"[1, (6 ounce) can, small shrimp, drained]","[1, (6 ounce) can, crabmeat, drained and flaked]","[1/2, cup, mayonnaise]","[1/4, cup thinly slice, d green onions]","[1/4, cup, diced celery]","[1, (8 ounce) package, shredded mozzarella che...","[1/8, teaspoon, salt]","[1/8, teaspoon, ground black pepper]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Senate Bean Soup from Idahoan®,"[1, (4 ounce) package, Idahoan® Roasted Garlic...","[2, tablespoons, olive oil]","[1, None, yellow onion, chopped]","[2, stalks, celery, chopped]","[2, None, carrots, chopped]","[2, (14.5 ounce) cans, white beans, rinsed and...","[1/2, bunch, parsley, chopped]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nutmeg Refrigerator Cookies,"[1/2, cup, butter]","[1/3, cup, white sugar]","[2/3, cup, packed brown sugar]","[1, None, egg]","[1, teaspoon, vanilla extract]","[1 1/2, cups, all-purpose flour]","[1/4, teaspoon, cream of tartar]","[1/4, teaspoon, salt]","[1/2, teaspoon, ground nutmeg]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,French Toast Sandwich,"[2, None, eggs]","[3, tablespoons, milk]","[3, tablespoons, white sugar]","[1/2, teaspoon, vanilla extract]","[1, cup, oil for frying]","[1, cup, milk, divided]","[2, tablespoons, custard powder]","[1/2, cup, confectioners' sugar]","[1/2, cup, whipped cream]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
type(df_csv['name'][0])

str

### Save to CSV

In [104]:
df_csv.to_csv('recipes_table_v2.csv', index=False, na_rep='')

In [106]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70881 entries, 0 to 70880
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          70881 non-null  object
 1   ingredient1   67286 non-null  object
 2   ingredient2   67835 non-null  object
 3   ingredient3   65070 non-null  object
 4   ingredient4   60441 non-null  object
 5   ingredient5   53979 non-null  object
 6   ingredient6   46660 non-null  object
 7   ingredient7   39072 non-null  object
 8   ingredient8   31828 non-null  object
 9   ingredient9   24985 non-null  object
 10  ingredient10  18892 non-null  object
 11  ingredient11  13849 non-null  object
 12  ingredient12  9875 non-null   object
 13  ingredient13  6884 non-null   object
 14  ingredient14  4750 non-null   object
 15  ingredient15  3180 non-null   object
 16  ingredient16  2103 non-null   object
 17  ingredient17  1354 non-null   object
 18  ingredient18  857 non-null    object
 19  ingr

# Now I need a new way of retrieving the appropriate ingredients

In [114]:
df_csv.iloc[0]

name                                    Death by Chocolate Mousse
ingredient1       [21, None, chocolate sandwich cookies, crushed]
ingredient2                          [1/4, cup, butter, softened]
ingredient3                                 [1, cup, heavy cream]
ingredient4     [1, (12 ounce) package, semisweet chocolate ch...
ingredient5                        [1, teaspoon, vanilla extract]
ingredient6                                      [1, pinch, salt]
ingredient7                                [2, cups, heavy cream]
ingredient8                               [1/4, cup, white sugar]
ingredient9                        [1, cup, heavy cream, chilled]
ingredient10                              [1/4, cup, white sugar]
ingredient11                                                  NaN
ingredient12                                                  NaN
ingredient13                                                  NaN
ingredient14                                                  NaN
ingredient

In [116]:
pd.notnull(df_csv.iloc[0])

name             True
ingredient1      True
ingredient2      True
ingredient3      True
ingredient4      True
ingredient5      True
ingredient6      True
ingredient7      True
ingredient8      True
ingredient9      True
ingredient10     True
ingredient11    False
ingredient12    False
ingredient13    False
ingredient14    False
ingredient15    False
ingredient16    False
ingredient17    False
ingredient18    False
ingredient19    False
ingredient20    False
ingredient21    False
ingredient22    False
ingredient23    False
ingredient24    False
ingredient25    False
ingredient26    False
ingredient27    False
ingredient28    False
ingredient29    False
ingredient30    False
Name: 0, dtype: bool

In [118]:
pd.notnull(df_csv.iloc[0])[1]

True

In [123]:
df_csv.columns[1]

'ingredient1'

In [124]:
df_csv.iloc[0][1]

['21', None, 'chocolate sandwich cookies, crushed']

In [125]:
# I can make a dictionary for the ingredients, but then what?
ingredient_dict = {}
for i in range(1, len(df_csv.iloc[0])):
    if pd.notnull(df_csv.iloc[0])[i]:
        ingredient_dict[df_csv.columns[i]] = df_csv.iloc[0][i]
        '''
        Instead of creating a dictionary here to store the ingredients for each returned recipe
        the ingredient counting could happen which would free up some processing and move more
        quickly through the whole loop.
        '''

ingredient_dict

{'ingredient1': ['21', None, 'chocolate sandwich cookies, crushed'],
 'ingredient2': ['1/4', 'cup', 'butter, softened'],
 'ingredient3': ['1', 'cup', 'heavy cream'],
 'ingredient4': ['1', '(12 ounce) package', 'semisweet chocolate chips'],
 'ingredient5': ['1', 'teaspoon', 'vanilla extract'],
 'ingredient6': ['1', 'pinch', 'salt'],
 'ingredient7': ['2', 'cups', 'heavy cream'],
 'ingredient8': ['1/4', 'cup', 'white sugar'],
 'ingredient9': ['1', 'cup', 'heavy cream, chilled'],
 'ingredient10': ['1/4', 'cup', 'white sugar']}

In [130]:
# Make a df to store results
results_df = pd.DataFrame(columns=['ingredients'])
results_df

,ingredients


In [131]:
# Add the ingredient_dict to this new df
results_df = results_df.append({'ingredients' :ingredient_dict}, ignore_index=True)
results_df

,ingredients
0,"{'ingredient1': ['21', None, 'chocolate sandwi..."


In [135]:
results_df['ingredients'][0]

{'ingredient1': ['21', None, 'chocolate sandwich cookies, crushed'],
 'ingredient2': ['1/4', 'cup', 'butter, softened'],
 'ingredient3': ['1', 'cup', 'heavy cream'],
 'ingredient4': ['1', '(12 ounce) package', 'semisweet chocolate chips'],
 'ingredient5': ['1', 'teaspoon', 'vanilla extract'],
 'ingredient6': ['1', 'pinch', 'salt'],
 'ingredient7': ['2', 'cups', 'heavy cream'],
 'ingredient8': ['1/4', 'cup', 'white sugar'],
 'ingredient9': ['1', 'cup', 'heavy cream, chilled'],
 'ingredient10': ['1/4', 'cup', 'white sugar']}

### This appears to work and will make it so much of the code below will be useable with minor modifications. It's only a matter of working out the query for the words entered and matching those with recipe names in the database.

In [153]:
# To use for this error: InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block

cursor = conn.cursor()

cursor.execute("""rollback;
""")

cursor.close()

In [144]:
%%capture
pip install psycopg2

In [145]:
# Connect to database.

import os
import psycopg2

conn = psycopg2.connect(database ='postgres', user = 'postgres', password = 'tz6MTgxObUZ62MNv0xgp', host = 'mydishdb-dev.c3und8sjo4p2.us-east-2.rds.amazonaws.com', port = '5432')

In [ ]:
# String comes in from frontend. Split the string into words. 

string = 'chicken noodle soup'
split_words = string.split()

In [178]:
cursor = conn.cursor()

command = """SELECT name 
                FROM recipes 
;
          """

cursor.execute(command)

name_table = cursor.fetchall()

cursor.close()

name_table[0]

('Death by Chocolate Mousse',)

In [164]:
cursor = conn.cursor()

command = """SELECT index, name 
                FROM recipes 
;
          """

cursor.execute(command)

test_table = cursor.fetchall()

cursor.close()

test_table[0]

(0, 'Death by Chocolate Mousse')

In [167]:
cursor = conn.cursor()

command = """SELECT index, name 
                FROM recipes
                WHERE index in (0, 2, 8)
;
          """

cursor.execute(command)

test = cursor.fetchall()

cursor.close()

test

[(0, 'Death by Chocolate Mousse'),
 (2, 'Senate Bean Soup from Idahoan®'),
 (8, "Emily's Famous Sloppy Joes")]

In [176]:
test[0][1]

'Death by Chocolate Mousse'

In [177]:
type(test[0][1])

str

In [173]:
test[0][2]

"['21', None, 'chocolate sandwich cookies, crushed']"

In [172]:
type(test[0][2])

str

In [174]:
string_to_list(test[0][2])

['21', None, 'chocolate sandwich cookies, crushed']

In [175]:
type(string_to_list(test[0][2]))

list

In [189]:
# Then query the recipe database to get recipe names that have matching words.

cursor = conn.cursor()

command = """SELECT name 
                FROM recipes 
                WHERE name ILIKE '%chicken%' AND
                    name ILIKE '%noodle%' AND
                    name ILIKE '%soup%'
          ;"""

cursor.execute(command)

table = cursor.fetchall()

cursor.close()

table


[('Gluten Free Chicken Noodle Soup',),
 ('Lemon Ginger Chicken Noodle Soup',),
 ('One-Step Chicken Noodle Soup',),
 ("Chef John's Homemade Chicken Noodle Soup",),
 ('Creamy Buffalo Chicken Noodle Soup',),
 ('Faux-kin Noodle Soup (Vegan Chicken Noodle Soup)',),
 ('Quick and Easy Chicken Noodle Soup',),
 ('NO YOLKS Chicken Noodle Soup',),
 ('Slow Cooker Chicken Noodle Soup',),
 ('Super Easy Chicken Noodle Soup',),
 ('Comforting Chicken Noodle Soup',),
 ('Never-Fail Chicken Noodle Soup',),
 ('Thai Chicken Noodle Soup',),
 ('Asian-Style Chicken Noodle Soup',),
 ('Home Made Chicken Noodle Soup!',),
 ("Mom Moak's Chicken Noodle Soup",),
 ('Grilled Chicken Noodle Soup',),
 ('Italian-Style Chicken Noodle Soup',),
 ('Thai Coconut Chicken Soup (Noodle Bowl)',),
 ('Instant Pot Lentil Chicken Noodle Soup',),
 ('Chicken Udon Noodle Soup',),
 ('Slow Cooker Chicken Vegetable Soup with Egg Noodles',),
 ('Sensational Chicken Noodle Soup',),
 ('Chicken Noodle Soup with Egg Noodles',),
 ('Hearty Panamani

In [191]:
cursor = conn.cursor()

command = """SELECT * 
                FROM recipes 
                WHERE name ILIKE '%chicken%' AND
                    name ILIKE '%noodle%' AND
                    name ILIKE '%soup%'
          ;"""

cursor.execute(command)

recipe_table = cursor.fetchall()

cursor.close()

recipe_table[0]


(2722,
 'Gluten Free Chicken Noodle Soup',
 "['4', 'tablespoons', 'extra-virgin olive oil']",
 "['1', None, 'large yellow onion, chopped']",
 "['1/2', 'cup', 'white wine']",
 "['2', '(10 ounce) bags', 'baby spinach']",
 "['1/2', 'pint', 'cherry tomatoes, quartered']",
 "['1/4', 'teaspoon', 'red pepper flakes']",
 "['1/2', 'teaspoon', 'black pepper']",
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [192]:
type(recipe_table)

list

In [193]:
type(recipe_table[0])

tuple

In [195]:
len(recipe_table[0])

32

In [196]:
len(recipe_table)

55

In [210]:
for i in range(len(recipe_table)):
    print(i, recipe_table[i][1])

0 Gluten Free Chicken Noodle Soup
1 Lemon Ginger Chicken Noodle Soup
2 One-Step Chicken Noodle Soup
3 Chef John's Homemade Chicken Noodle Soup
4 Creamy Buffalo Chicken Noodle Soup
5 Faux-kin Noodle Soup (Vegan Chicken Noodle Soup)
6 Quick and Easy Chicken Noodle Soup
7 NO YOLKS Chicken Noodle Soup
8 Slow Cooker Chicken Noodle Soup
9 Super Easy Chicken Noodle Soup
10 Comforting Chicken Noodle Soup
11 Never-Fail Chicken Noodle Soup
12 Thai Chicken Noodle Soup
13 Asian-Style Chicken Noodle Soup
14 Home Made Chicken Noodle Soup!
15 Mom Moak's Chicken Noodle Soup
16 Grilled Chicken Noodle Soup
17 Italian-Style Chicken Noodle Soup
18 Thai Coconut Chicken Soup (Noodle Bowl)
19 Instant Pot Lentil Chicken Noodle Soup
20 Chicken Udon Noodle Soup
21 Slow Cooker Chicken Vegetable Soup with Egg Noodles
22 Sensational Chicken Noodle Soup
23 Chicken Noodle Soup with Egg Noodles
24 Hearty Panamanian-Style Chicken Noodle Soup (Sancocho)
25 Grandma's Chicken Noodle Soup
26 The Ultimate Chicken Noodle So

In [212]:
string_to_list(recipe_table[0][2])

['4', 'tablespoons', 'extra-virgin olive oil']

In [216]:
string_to_list(recipe_table[0][2])[2]

'extra-virgin olive oil'

In [238]:
# Count instances of each ingredient to find most common.

# Initialize a Counter for tabulating how often each ingredient occurs
ingredient_counts = Counter()

# Count each instance of each ingredient
for i in range(len(recipe_table)):
    for j in range(2, len(recipe_table[i])):
        if recipe_table[i][j]:
            ingredient = string_to_list(recipe_table[i][j])[2]
            ingredient_counts.update({ingredient: 1})

ingredient_counts

Counter({'extra-virgin olive oil': 1,
         'large yellow onion, chopped': 1,
         'white wine': 3,
         'baby spinach': 1,
         'cherry tomatoes, quartered': 1,
         'red pepper flakes': 1,
         'black pepper': 2,
         'olive oil': 6,
         'd': 27,
         'medium onion, chopped': 4,
         'minced fresh ginger root': 3,
         'cloves garlic, minced': 7,
         'shredded cooked chicken': 3,
         'lemon juice': 2,
         'chopped fresh parsley': 4,
         'frozen peas (optional)': 1,
         'ketchup': 1,
         'butter': 7,
         'diced carrot': 1,
         'diced onion': 2,
         'diced celery': 3,
         'salt': 12,
         'cayenne pepper (optional)': 1,
         'salt and ground black pepper to taste': 1,
         'extra-wide egg noodles': 1,
         'celery, chopped': 5,
         'carrots, peeled and chopped': 1,
         'clove garlic, chopped': 1,
         'flour': 1,
         ') carton Swanson&reg; Chicken Broth': 2,


In [219]:
# Loop through most common to save quantity and measurement to get most common of those.

# Get the top 30 ingredients sorted by most common
top_30 = sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True)[:30]

# Get the ingredients that occured in at least 25% of recipes returned
above_25_percent = [(tup[0], round(100*tup[1]/len(recipe_table), 1)) for tup in top_30 if 100*tup[1]/len(recipe_table) >= 25]

above_25_percent

[('d', 49.1),
 ('chicken broth', 36.4),
 ('', 34.5),
 ('water', 30.9),
 ('egg noodles', 25.5)]

In [227]:
for item in above_25_percent:
    print(item[0])

d
chicken broth

water
egg noodles


In [230]:
# for i in range(len(recipe_table)):
        
#     for j in range(2, len(recipe_table[i])):
        
#         if recipe_table[i][j]:
            
#             print(string_to_list(recipe_table[i][j])[2])

In [234]:
# Create dictionary of information. Turn into dictionary (then JSON) and return.
results_dict = {}

# Get the ingredient information and put it in a dictionary
for item in above_25_percent:
    quantity_list = []
    unit_list = []

    for i in range(len(recipe_table)):
        
        for j in range(2, len(recipe_table[i])):

            if recipe_table[i][j]:
                if string_to_list(recipe_table[i][j])[2] == item[0]:
                    #print(recipe_table[i][j])

                    quantity = string_to_list(recipe_table[i][j])[0]
                    unit = string_to_list(recipe_table[i][j])[1]

                    quantity_list.append(quantity)
                    unit_list.append(unit)
                    
#                     print(quantity)

    # Getting and saving the most common quantity and unit for each ingredient
    data = Counter(quantity_list)
    quantity = data.most_common(1)
    data = Counter(unit_list)
    unit = data.most_common(1)

#     print(quantity)
    results_dict[item] = {'quantity': quantity[0][0], 
                          'unit': unit[0][0], 
                          'ingredient': item[0]}

results_dict

{('d', 49.1): {'quantity': '2',
  'unit': 'stalks celery, slice',
  'ingredient': 'd'},
 ('chicken broth', 36.4): {'quantity': '4',
  'unit': 'cups',
  'ingredient': 'chicken broth'},
 ('', 34.5): {'quantity': '1', 'unit': 'bay leaf', 'ingredient': ''},
 ('water', 30.9): {'quantity': '8', 'unit': 'cups', 'ingredient': 'water'},
 ('egg noodles', 25.5): {'quantity': '1',
  'unit': 'cups',
  'ingredient': 'egg noodles'}}

In [235]:
conn.close()

## Put It All Together

In [ ]:
%%capture
pip install psycopg2

In [ ]:
import ast
import psycopg2
from collections import Counter

In [241]:
def string_to_list(x):
    return ast.literal_eval(x)


def ingredient_getter(word):
    
    results_dict = {}
    split_words = word.split()
    
    conn = psycopg2.connect(database ='postgres', user = 'postgres', password = 'tz6MTgxObUZ62MNv0xgp', host = 'mydishdb-dev.c3und8sjo4p2.us-east-2.rds.amazonaws.com', port = '5432')
    cursor = conn.cursor()

    command = f"SELECT * FROM recipes WHERE name ILIKE '%{split_words[0]}%' "
    if len(split_words) > 1:
        for i in range(1, len(split_words)):
            command += f"AND name ILIKE '%{split_words[i]}%' "
    command += ";"

    cursor.execute(command)

    recipe_table = cursor.fetchall()

    cursor.close()
    conn.close()
    
    # Initialize a Counter for tabulating how often each ingredient occurs
    ingredient_counts = Counter()

    # Count each instance of each ingredient
    for i in range(len(recipe_table)):
        for j in range(2, len(recipe_table[i])):
            if recipe_table[i][j]:
                ingredient = string_to_list(recipe_table[i][j])[2]
                ingredient_counts.update({ingredient: 1})
    
    # Get the top 30 ingredients sorted by most common
    top_30 = sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True)[:30]

    # Get the ingredients that occured in at least 25% of recipes returned
    above_25_percent = [(tup[0], round(100*tup[1]/len(recipe_table), 1)) for tup in top_30 if 100*tup[1]/len(recipe_table) >= 25]

    # Get the ingredient information and put it in a dictionary
    for item in above_25_percent:
        quantity_list = []
        unit_list = []

        for i in range(len(recipe_table)):

            for j in range(2, len(recipe_table[i])):

                if recipe_table[i][j]:
                    if string_to_list(recipe_table[i][j])[2] == item[0]:

                        quantity = string_to_list(recipe_table[i][j])[0]
                        unit = string_to_list(recipe_table[i][j])[1]

                        quantity_list.append(quantity)
                        unit_list.append(unit)


        # Getting and saving the most common quantity and unit for each ingredient
        data = Counter(quantity_list)
        quantity = data.most_common(1)
        data = Counter(unit_list)
        unit = data.most_common(1)

        results_dict[item] = {'quantity': quantity[0][0], 
                              'unit': unit[0][0], 
                              'ingredient': item[0]}

    return results_dict
    

In [244]:
ingredient_getter('brownies')

{('vanilla extract', 75.6): {'quantity': '1',
  'unit': 'teaspoon',
  'ingredient': 'vanilla extract'},
 ('white sugar', 66.6): {'quantity': '1',
  'unit': 'cup',
  'ingredient': 'white sugar'},
 ('all-purpose flour', 65.8): {'quantity': '1',
  'unit': 'cup',
  'ingredient': 'all-purpose flour'},
 ('salt', 60.7): {'quantity': '1/4', 'unit': 'teaspoon', 'ingredient': 'salt'},
 ('eggs', 51.4): {'quantity': '2', 'unit': None, 'ingredient': 'eggs'},
 ('baking powder', 36.0): {'quantity': '1',
  'unit': 'teaspoon',
  'ingredient': 'baking powder'},
 ('unsweetened cocoa powder', 35.7): {'quantity': '1',
  'unit': 'cup',
  'ingredient': 'unsweetened cocoa powder'},
 ('butter', 30.8): {'quantity': '1/2', 'unit': 'cup', 'ingredient': 'butter'}}

In [245]:
import time
start_time = time.time()
ingredient_getter('brownies')
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.3657753467559814 seconds ---
